<a href="https://colab.research.google.com/github/jespimentel/aposentaveis/blob/main/aposentaveis_nov_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import unicodedata
import difflib

import folium
from folium.plugins import MarkerCluster

In [2]:
membros = '/content/drive/MyDrive/Datasets/abono_novembro_24.xlsx'

# Fonte dos dados georreferenciados: https://github.com/alanwillms/geoinfo/
coordenadas_municipios = '/content/drive/MyDrive/Datasets/latitude-longitude-cidades.csv'

df_membros_ativos = pd.read_excel(membros, index_col=0)
df_coordenadas = pd.read_csv(coordenadas_municipios, sep=';', index_col=0)

In [3]:
def remover_acentos(text):
  return ''.join(c for c in unicodedata.normalize('NFD', text)
                  if unicodedata.category(c) != 'Mn')

In [4]:
def encontra_municipio(coluna):
    match coluna:
        case coluna if "PROCURADOR" in coluna:
            return 'SAO PAULO'
        case coluna if 'CAPITAL' in coluna:
            return 'SAO PAULO'
        case coluna if 'MILITAR' in coluna:
            return 'SAO PAULO'
        case coluna if 'MANDADOS' in coluna:
            return 'SAO PAULO'
        case coluna if 'HABITACAO' in coluna:
            return 'SAO PAULO'
        case coluna if 'EXECUCOES' in coluna:
          return 'SAO PAULO'
        case coluna if 'FALENCIAS' in coluna:
          return 'SAO PAULO'
        case coluna if 'HUMANOS' in coluna:
          return 'SAO PAULO'
        case coluna if 'PENHA DE FRANCA' in coluna:
          return 'SAO PAULO'
        case coluna if 'NOSSA SENHORA' in coluna:
          return 'SAO PAULO'
        case coluna if 'FORO REGIONAL' in coluna:
          return 'SAO PAULO'
        case coluna if 'PROMOTORIA DE JUSTICA REGIONAL' in coluna:
          return 'SAO PAULO'
        case coluna if 'PROMOTORIA DE JUSTICA DE DISTRITAL DE ' in coluna:
          return coluna.split('PROMOTORIA DE JUSTICA DE DISTRITAL DE ')[-1]
        case coluna if 'PROMOTORIA DE JUSTICA DE ' in coluna:
          return coluna.split('PROMOTORIA DE JUSTICA DE ')[-1]
        case coluna if 'PROMOTORIAS DE JUSTICA DE AREA REGIONAL DE ' in coluna:
          return coluna.split('PROMOTORIAS DE JUSTICA DE AREA REGIONAL DE ')[-1]
        case coluna if 'PROMOTORIAS DE JUSTICA DA AREA REGIONAL DE ' in coluna:
          return coluna.split('PROMOTORIAS DE JUSTICA DA AREA REGIONAL DE ')[-1]
        case coluna if 'PROMOTORIA DE JUSTICA DISTRITAL DE ' in coluna:
          return coluna.split('PROMOTORIA DE JUSTICA DISTRITAL DE ')[-1]
        case coluna if 'IACANGA' in coluna:
          return 'IACANGA'

        case _:
            return 'SAO PAULO'

In [5]:
def encontra_semelhante(texto, lista_referencia):
  try:
    return difflib.get_close_matches(texto, lista_referencia, 1, 0.3)[0]
  except IndexError:
    return 'NAO ENCONTRADO'

In [6]:
def gera_mapa_clusterizado(df, cores_dict):
  COORDENADAS_MEDIAS = df['latitude'].mean(), df['longitude'].mean()
  # Cria o mapa vazio
  mapa = folium.Map(location=COORDENADAS_MEDIAS, zoom_start=7)

  # Cria um MarkerCluster para agrupar os marcadores
  marker_cluster = MarkerCluster().add_to(mapa)

  # Cria um dicionário de cores para os marcadores
  cores = cores_dict

  # Cor padrão para naturezas não especificadas
  cor_padrao = 'beige'

  # Adiciona os marcadores
  for _, row in df.iterrows():

    html_popup = f"""
    <strong>{row['Cargo']}</strong> <br>
    {row['Lotação']}<br>
    """
    # Determina a cor do marcador com base na natureza
    cor = cores.get(row['Cargo'], cor_padrao)

    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=html_popup,
        icon=folium.Icon(color=cor, icon='info-sign')
    ).add_to(marker_cluster)

  # Adiciona o título ao mapa
  titulo_html = '''
              <h3 align="center" style="font-size:20px"><b>Onde estão os colegas que já podem se aposentar? (ref.: Nov. 2024)</b></h3>
              '''
  mapa.get_root().html.add_child(folium.Element(titulo_html))

  # Adiciona o rodapé ao mapa
  rodape_html = '''
              <p align="center" style="font-size:12px;color:gray;"><i>github.com/jespimentel/aposentaveis</i></p>
              '''
  mapa.get_root().html.add_child(folium.Element(rodape_html))

  return mapa

In [7]:
# Adequação do dataframe de membros ativos
df_membros_ativos['Lotação'] = df_membros_ativos['Lotação'].str.upper()
df_membros_ativos['Lotação'] = df_membros_ativos['Lotação'].apply(remover_acentos)
df_membros_ativos['Município'] = df_membros_ativos['Lotação'].apply(encontra_municipio)

df_membros_ativos[['Cargo', 'Lotação', 'Município']].sample(5)

,Cargo,Lotação,Município
Matrícula,,,
2770,PROMOTOR DE JUSTICA (ENTRANCIA FINAL),PROMOTORIA DE JUSTICA DA CAPITAL,SAO PAULO
3026,PROMOTOR DE JUSTICA (ENTRANCIA FINAL),PROMOTORIA DE JUSTICA DA CAPITAL,SAO PAULO
2538,PROMOTOR DE JUSTICA (ENTRANCIA FINAL),PROMOTORIA DE JUSTICA DA CAPITAL,SAO PAULO
2708,PROMOTOR DE JUSTICA (ENTRANCIA INTERMEDIARIA),PROMOTORIAS DE JUSTICA DA AREA REGIONAL DE SOR...,SOROCABA
698587,PROCURADOR DE JUSTICA,PROCURADORIA DE JUSTICA CRIMINAL,SAO PAULO


In [8]:
# Cria o dataframe com as coordenadas dos municípios do Estado de SP
df_coordenadas_sp = df_coordenadas.loc[df_coordenadas['uf'] == 'SP'].copy()

# Adequação do dataframe de coordenadas
df_coordenadas_sp['municipio'] = df_coordenadas_sp['municipio'].str.upper()
df_coordenadas_sp['municipio'] = df_coordenadas_sp['municipio'].apply(remover_acentos)
df_coordenadas_sp = df_coordenadas_sp.drop(columns=['uf'])
df_coordenadas_sp.sample(5)

,municipio,longitude,latitude
id_municipio,,,
4819,BRAUNA,-50.317489,-21.499039
5330,TAPIRATIBA,-46.744825,-21.471324
4997,ITAPURA,-51.506322,-20.641911
5249,SANTA CLARA D`OESTE,-50.949109,-20.090041
4901,EMBU-GUACU,-46.813593,-23.829717


In [9]:
lista_referencia = df_coordenadas_sp['municipio'].to_list()
lista_referencia = list(set(lista_referencia))

df_membros_ativos['municipio'] = df_membros_ativos['Município'].apply(lambda x: encontra_semelhante(x, lista_referencia))

In [10]:
# Cria o dataframe de membros com requisitos para se aposentar
df_aposentaveis = df_membros_ativos.loc[df_membros_ativos['Abono'] != ' - '].drop(columns=['Abono'])

In [11]:
print('\nPodem se aposentar:')
display(df_aposentaveis.groupby('Cargo')['Município'].count().sort_values(ascending=False))


Podem se aposentar:


,Município
Cargo,
PROCURADOR DE JUSTICA,235
PROMOTOR DE JUSTICA (ENTRANCIA FINAL),186
PROMOTOR DE JUSTICA (ENTRANCIA INTERMEDIARIA),9
PROMOTOR DE JUSTICA (ENTRANCIA INICIAL),2
PROCURADOR GERAL DE JUSTICA,1


In [12]:
# Fazendo o merge dos DataFrames
df_merged = pd.merge(df_aposentaveis, df_coordenadas_sp, on='municipio')
df_merged = df_merged[['Cargo', 'Lotação', 'Município', 'latitude', 'longitude']]

display(df_merged)

,Cargo,Lotação,Município,latitude,longitude
0,PROCURADOR DE JUSTICA,PROCURADORIA DE JUSTICA CRIMINAL,SAO PAULO,-23.532905,-46.639520
1,PROMOTOR DE JUSTICA (ENTRANCIA FINAL),PROMOTORIA DE JUSTICA DE OURINHOS,OURINHOS,-22.979695,-49.869702
2,PROCURADOR DE JUSTICA,PROCURADORIA JUSTICA HABEAS CORPUS MANDADOS SE...,SAO PAULO,-23.532905,-46.639520
3,PROMOTOR DE JUSTICA (ENTRANCIA FINAL),PROMOTORIA DE JUSTICA REGIONAL DE SANTANA,SAO PAULO,-23.532905,-46.639520
4,PROMOTOR DE JUSTICA (ENTRANCIA FINAL),PROMOTORIA DE JUSTICA DA CAPITAL,SAO PAULO,-23.532905,-46.639520
...,...,...,...,...,...
428,PROMOTOR DE JUSTICA (ENTRANCIA FINAL),PROMOTORIA DE JUSTICA DE OSASCO,OSASCO,-23.532390,-46.791555
429,PROMOTOR DE JUSTICA (ENTRANCIA FINAL),PROMOTORIA DE JUSTICA DE JALES,JALES,-20.267227,-50.549442
430,PROMOTOR DE JUSTICA (ENTRANCIA FINAL),PROMOTORIA DE JUSTICA DA CAPITAL,SAO PAULO,-23.532905,-46.639520
431,PROCURADOR DE JUSTICA,PROCURADORIA DE JUSTICA CRIMINAL,SAO PAULO,-23.532905,-46.639520


In [13]:
print(df_merged['Cargo'].unique())

cores_dict = {
      'PROCURADOR DE JUSTICA': 'black',
      'PROCURADOR GERAL DE JUSTICA': 'black',
      'PROMOTOR DE JUSTICA (ENTRANCIA FINAL)': 'darkred',
      'PROMOTOR DE JUSTICA (ENTRANCIA INTERMEDIARIA)': 'blue',
      'PROMOTOR DE JUSTICA (ENTRANCIA INICIAL)': 'green'
  }

['PROCURADOR DE JUSTICA' 'PROMOTOR DE JUSTICA (ENTRANCIA FINAL)'
 'PROMOTOR DE JUSTICA (ENTRANCIA INICIAL)'
 'PROMOTOR DE JUSTICA (ENTRANCIA INTERMEDIARIA)'
 'PROCURADOR GERAL DE JUSTICA']


In [14]:
# Gerando o mapa clusterizado
mapa = gera_mapa_clusterizado(df_merged, cores_dict)
mapa.save('index.html')
display(mapa)